In [4]:
import requests
from sklearn.preprocessing import PolynomialFeatures

def fuelConversion(fuel):
    g = 3.785411784   # liters per gallon
    m = 1609.344    # meters per mile
    kpg = (g / m * 1000)    # kilometers per gallon 
    return  100 / fuel * kpg    # returns fuel as mpg to l/100km or vice versa

def predict_fuel_consumption_l_100_km(weight_kg, year):
  weight_lbs = weight_kg * 2.205
  year_mod_100 = year % 100

  input = [
      [weight_lbs, # Gewicht in Pfund (1110 kg)
      year_mod_100]   # Baujahr Modulo 100 (Produktionsbeginn 1971)
      ]

  transformed_input = PolynomialFeatures(degree=2).fit_transform(input).tolist()
  inference_input = {
    'instances': transformed_input
  }

  response = requests.post(
    "http://<Public Kubeflow Endpoint>/kserve/v1/models/sklearn-mpg:predict", 
    json=inference_input, 
    headers={"Host": "sklearn-mpg.kserve-deploy-test.example.com"}
    )

  mpg = response.json()["predictions"][0]
  return fuelConversion(mpg)

In [5]:
# Audi 60L: 
baujahr = 1971
gewicht = 970 + 110 # 4-Türer (970kg) + Fahrer mit Gepäck (110kg)

predict_fuel_consumption_l_100_km(weight_kg=gewicht, year=baujahr)

10.042277210575556

In [6]:
# Audi A4: 
baujahr = 2023
gewicht = 1595 # Leergewicht + Fahrer

predict_fuel_consumption_l_100_km(weight_kg=gewicht, year=baujahr)

1.8032310948108592